In [8]:
import json
from gerrychain import Graph

file_path = r"C:\Users\darbz\Downloads\districting-data-2020-county\districting-data-2020-county\KS_county.json"

G = Graph.from_json(file_path)

In [9]:
for node in G.nodes:
    county_name = G.nodes[node]['NAME20']
    county_population = G.nodes[node]['P0010001']
    G.nodes[node]['TOTPOP'] = county_population
    print("Node",node,"represents",county_name,"County, with a population of",county_population,"in the 2020 census.")

Node 0 represents Montgomery County, with a population of 31486 in the 2020 census.
Node 1 represents Finney County, with a population of 38470 in the 2020 census.
Node 2 represents Kearny County, with a population of 3983 in the 2020 census.
Node 3 represents Ford County, with a population of 34287 in the 2020 census.
Node 4 represents Chase County, with a population of 2572 in the 2020 census.
Node 5 represents Lyon County, with a population of 32179 in the 2020 census.
Node 6 represents Atchison County, with a population of 16348 in the 2020 census.
Node 7 represents Barton County, with a population of 25493 in the 2020 census.
Node 8 represents Ellis County, with a population of 28934 in the 2020 census.
Node 9 represents Reno County, with a population of 61898 in the 2020 census.
Node 10 represents Leavenworth County, with a population of 81881 in the 2020 census.
Node 11 represents Johnson County, with a population of 609863 in the 2020 census.
Node 12 represents Wyandotte County

In [10]:
deviation = 0.01

k = 4
total_population = sum( G.nodes[i]['TOTPOP'] for i in G.nodes )
ideal_population = total_population / k

import math
Lower_bound = math.ceil( ( 1 - deviation / 2 ) * ideal_population )
Upper_bound = math.floor( ( 1 + deviation / 2 ) * ideal_population )
print("Using Lower Bound =",Lower_bound,"and Upper Bound =",Upper_bound,"and # Districts =",k)

Using Lower Bound = 730798 and Upper Bound = 738142 and # Districts = 4


In [11]:
import gurobipy as gp
from gurobipy import GRB

m = gp.Model()

# variables
x = m.addVars( G.nodes, k, vtype=GRB.BINARY ) # x[i,j] = 1 if county i is assigned to district j
y = m.addVars( G.edges, vtype=GRB.BINARY )    # y[u,v] = 1 when edge {u,v} is cut

m.setObjective( gp.quicksum( y[u,v] for u,v in G.edges ), GRB.MINIMIZE )

In [12]:
# constraint that each county i is assigned to one district j
m.addConstrs( gp.quicksum( x[i,j] for j in range(k) ) == 1 for i in G.nodes )

# constraint that each district j has a population at least LB and at most UB
m.addConstrs( gp.quicksum( G.nodes[i]['TOTPOP'] * x[i,j] for i in G.nodes ) >= Lower_bound for j in range(k) )
m.addConstrs( gp.quicksum( G.nodes[i]['TOTPOP'] * x[i,j] for i in G.nodes ) <= Upper_bound for j in range(k) )

# constraint that edge {u,v} is cut if u is assigned to district j but v is not
m.addConstrs( x[u,j] - x[v,j] <= y[u,v] for u,v in G.edges for j in range(k) )

m.update()

In [13]:
# root variables r[i,j] = 1 if county (node) i is the "root" of district j
r = m.addVars( G.nodes, k, vtype=GRB.BINARY )

import networkx as nx
DG = nx.DiGraph(G)     

f = m.addVars( DG.edges )
M = G.number_of_nodes() - k + 1

# each district j should have one root
m.addConstrs( gp.quicksum( r[i,j] for i in G.nodes ) == 1 for j in range(k) )

# if node i is not assigned to district j, then it cannot be its root
m.addConstrs( r[i,j] <= x[i,j] for i in G.nodes for j in range(k) ) 

# if not a root, consume some flow.
# if a root, only send out (so much) flow.
m.addConstrs( gp.quicksum( f[j,i] - f[i,j] for j in G.neighbors(i) ) 
             >= 1 - M * gp.quicksum( r[i,j] for j in range(k) ) for i in G.nodes )

# do not send flow across cut edges
m.addConstrs( f[i,j] + f[j,i] <= M * ( 1 - y[i,j] ) for i,j in G.edges )

m.update()
m.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1957 rows, 1629 columns and 7937 nonzeros
Model fingerprint: 0x745a7974
Variable types: 526 continuous, 1103 integer (1103 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+05]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+05]
Presolve time: 0.04s
Presolved: 1957 rows, 1629 columns, 7933 nonzeros
Variable types: 526 continuous, 1103 integer (1103 binary)

Root relaxation: objective 7.500000e-01, 803 iterations, 0.05 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.75000    0  437          -    0.75000      -     -    0s
     0     0 

In [14]:
print("The number of cut edges is",m.objval)

# retrieve the districts and their populations
districts = [ [ i for i in G.nodes if x[i,j].x > 0.5 ] for j in range(k) ]
district_county_names = [ [ G.nodes[i]['NAME20'] for i in district ] for district in districts ]
district_populations = [ sum( G.nodes[i]['TOTPOP'] for i in district ) for district in districts ]

for j in range(k):
    print("District",j,"has these nodes =",districts[j],"and this population =",district_populations[j] )
    print("The corresponding county names are =",district_county_names[j] )
    print("")

The number of cut edges is 33.0
District 0 has these nodes = [0, 22, 23, 33, 35, 36, 38, 50, 62, 73, 91, 104] and this population = 735509
The corresponding county names are = ['Montgomery', 'Labette', 'Crawford', 'Sedgwick', 'Sumner', 'Cowley', 'Bourbon', 'Wilson', 'Elk', 'Chautauqua', 'Cherokee', 'Neosho']

District 1 has these nodes = [6, 10, 12, 15, 17, 18, 24, 27, 29, 31, 37, 63, 70] and this population = 731404
The corresponding county names are = ['Atchison', 'Leavenworth', 'Wyandotte', 'Douglas', 'Riley', 'Pottawatomie', 'Doniphan', 'Jefferson', 'Jackson', 'Shawnee', 'Brown', 'Nemaha', 'Marshall']

District 2 has these nodes = [1, 2, 3, 4, 5, 7, 8, 9, 16, 19, 20, 25, 26, 28, 32, 34, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 64, 66, 67, 68, 71, 72, 74, 75, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103] and this population = 737707
The corresponding county names are = ['Finney', 'Ke

In [17]:
import networkx as nx
for district in districts:
    print("Is district =", district, "connected?", nx.is_connected( G.subgraph( district ) ) )

Is district = [0, 22, 23, 33, 35, 36, 38, 50, 62, 73, 91, 104] connected? True
Is district = [6, 10, 12, 15, 17, 18, 24, 27, 29, 31, 37, 63, 70] connected? True
Is district = [1, 2, 3, 4, 5, 7, 8, 9, 16, 19, 20, 25, 26, 28, 32, 34, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 64, 66, 67, 68, 71, 72, 74, 75, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103] connected? True
Is district = [11, 13, 14, 21, 30, 45, 65, 69, 76, 90] connected? True


In [21]:
from PIL import Image

image = Image.open(r"C:\Users\darbz\Downloads\map-image.png")
image.show()